In [7]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter('ignore')

# Generate synthetic dataset
np.random.seed(42)
X = np.random.rand(100, 1)  # Feature
y = 2 * X + np.random.normal(0, 0.2, (100, 1))  # Target

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define quantiles to predict
quantiles = [0.1, 0.5, 0.9]

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define LightGBM parameters for quantile regression
params = {
    'objective': 'quantile',  # Quantile regression objective
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'verbose': -1
}

# Initialize a dictionary to store quantile models
quantile_models = {}

# Train a quantile regression model for each quantile
for q in quantiles:
    params['alpha'] = q
    model = lgb.train(params, train_data, valid_sets=[test_data], early_stopping_rounds=20)
    quantile_models[q] = model

# Predict quantiles for the test set
quantile_preds = np.column_stack([quantile_models[q].predict(X_test) for q in quantiles])

# Calculate mean absolute error for each quantile
mae_by_quantile = [mean_absolute_error(y_test, quantile_preds[:, i]) for i, q in enumerate(quantiles)]
for q, mae in zip(quantiles, mae_by_quantile):
    print(f"Mean Absolute Error for Quantile {q:.1f}: {mae:.4f}")


[1]	valid_0's quantile: 0.0785843
Training until validation scores don't improve for 20 rounds
[2]	valid_0's quantile: 0.0762615
[3]	valid_0's quantile: 0.0740549
[4]	valid_0's quantile: 0.0719586
[5]	valid_0's quantile: 0.0699671
[6]	valid_0's quantile: 0.0680752
[7]	valid_0's quantile: 0.0662779
[8]	valid_0's quantile: 0.0645704
[9]	valid_0's quantile: 0.0629484
[10]	valid_0's quantile: 0.0614074
[11]	valid_0's quantile: 0.0604489
[12]	valid_0's quantile: 0.0595384
[13]	valid_0's quantile: 0.0586734
[14]	valid_0's quantile: 0.0573746
[15]	valid_0's quantile: 0.0565674
[16]	valid_0's quantile: 0.0558006
[17]	valid_0's quantile: 0.0550721
[18]	valid_0's quantile: 0.0539758
[19]	valid_0's quantile: 0.0532963
[20]	valid_0's quantile: 0.0526508
[21]	valid_0's quantile: 0.0516846
[22]	valid_0's quantile: 0.0510786
[23]	valid_0's quantile: 0.0498628
[24]	valid_0's quantile: 0.0487078
[25]	valid_0's quantile: 0.0476105
[26]	valid_0's quantile: 0.0465681
[27]	valid_0's quantile: 0.0456787
[28

In [8]:
quantile_preds = np.column_stack([quantile_models[q].predict(X_test) for q in quantiles])

In [9]:
quantile_preds

array([[0.05779127, 0.31243681, 0.71921451],
       [1.2967112 , 1.62567021, 1.99476642],
       [1.2967112 , 1.62567021, 1.99476642],
       [0.74407493, 1.33162142, 1.342029  ],
       [0.2805286 , 0.45153001, 0.71921451],
       [0.62311254, 0.78635896, 1.03417358],
       [0.2805286 , 0.5279159 , 0.71921451],
       [1.2967112 , 1.62567021, 1.99476642],
       [0.05779127, 0.31243681, 0.71921451],
       [0.4678602 , 0.75081928, 0.95823492],
       [0.62311254, 0.78635896, 1.03417358],
       [0.74407493, 1.04950589, 1.342029  ],
       [1.2967112 , 1.62567021, 1.99476642],
       [1.2967112 , 1.62567021, 1.99476642],
       [0.05779127, 0.31243681, 0.71921451],
       [0.05779127, 0.31243681, 0.71921451],
       [1.2967112 , 1.62567021, 1.99476642],
       [0.05779127, 0.31243681, 0.71921451],
       [1.2967112 , 1.62567021, 1.99476642],
       [0.05779127, 0.31243681, 0.71921451]])